### 前準備

In [2]:
#{コード3.1 3章の準備}
import pandas as pd
import numpy as np 
import os 
from glob import glob

### レコード操作

In [3]:
# {コード3.17 3.4節の準備}
df_rec1 = pd.read_csv('./in/3_company_data.csv',dtype = {'所在地_市区郡コード':'object'}) 

print('df_rec1'); display(df_rec1.head())
print(len(df_rec1))

df_rec1
1084


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率
0,A08179573,01_北海道,167,2018,306,-0.0001
1,A08179573,NaN,167,2019,302,0.0021
2,A08507655,NaN,0,2018,270,0.0040
3,A08507655,NaN,0,2019,278,0.0026
4,A08514558,NaN,235,2018,332,0.0031


In [3]:
# {コード3.18 直前の値でnull を埋める}
df_rec2 = df_rec1.copy()

df_rec2['所在地_都道府県_補完']=df_rec2['所在地_都道府県'].fillna(method='ffill')

print('df_rec2'); display(df_rec2.head())
print(len(df_rec2))

df_rec2


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率,所在地_都道府県_補完
0,A08179573,01_北海道,167,2018,306,-0.0001,01_北海道
1,A08179573,NaN,167,2019,302,0.0021,01_北海道
2,A08507655,NaN,0,2018,270,0.0040,01_北海道
3,A08507655,NaN,0,2019,278,0.0026,01_北海道
4,A08514558,NaN,235,2018,332,0.0031,01_北海道


1084


In [4]:
# {コード3.19 データフレームの並び替え}
df_rec3 = df_rec2.sort_values(['企業管理コード','決算期年'],ascending=[True,True])

print('df_rec3'); display(df_rec3.head())
print(len(df_rec3))

df_rec3


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率,所在地_都道府県_補完
802,A00115478,29_奈良県,102,2018,337,0.0024,29_奈良県
803,A00115478,NaN,102,2019,258,0.0045,29_奈良県
920,A00116790,38_愛媛県,91,2018,335,0.0034,38_愛媛県
921,A00116790,NaN,91,2019,315,0.0052,38_愛媛県
936,A03112202,40_福岡県,88,2018,271,0.0045,40_福岡県


1084


In [5]:
# {コード3.20 レコードの重複削除}
df_rec4 = df_rec3[~df_rec3[['企業管理コード','決算期年']].duplicated()]    # 重複行の抽出

print('df_rec4'); display(df_rec4.head())
print(len(df_rec4))

df_rec4


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率,所在地_都道府県_補完
802,A00115478,29_奈良県,102,2018,337,0.0024,29_奈良県
803,A00115478,NaN,102,2019,258,0.0045,29_奈良県
920,A00116790,38_愛媛県,91,2018,335,0.0034,38_愛媛県
921,A00116790,NaN,91,2019,315,0.0052,38_愛媛県
936,A03112202,40_福岡県,88,2018,271,0.0045,40_福岡県


1034


In [6]:
# {コード3.21 前行との差分を計算する}
df_rec5 = df_rec4.copy()

# 企業管理コードごとに前年差の計算
df_rec5["売上高前年差_百万円"] = df_rec5.groupby(["企業管理コード"])["売上高_百万円"].diff()

# 企業管理コードごとに変化量の計算(b-a)/a 
df_rec5["売上高前年変化量"] = df_rec5.groupby(["企業管理コード"])["売上高_百万円"].pct_change()

df_rec5["売上高前年比パーセント"] = (df_rec5["売上高前年変化量"]+1)*100


print('df_rec5'); display(df_rec5.head())
print(len(df_rec5))

df_rec5


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率,所在地_都道府県_補完,売上高前年差_百万円,売上高前年変化量,売上高前年比パーセント
802,A00115478,29_奈良県,102,2018,337,0.0024,29_奈良県,NaN,NaN,NaN
803,A00115478,NaN,102,2019,258,0.0045,29_奈良県,-79.0,-0.234421,76.557864
920,A00116790,38_愛媛県,91,2018,335,0.0034,38_愛媛県,NaN,NaN,NaN
921,A00116790,NaN,91,2019,315,0.0052,38_愛媛県,-20.0,-0.059701,94.029851
936,A03112202,40_福岡県,88,2018,271,0.0045,40_福岡県,NaN,NaN,NaN


1034


In [7]:
# {コード3.22 首都圏の企業の抽出}
df_rec6_1=df_rec4[df_rec4['所在地_都道府県_補完'].isin (['11_埼玉県', '12_千葉県','13_東京都','14_神奈川県' ])]

print('df_rec6_1'); display(df_rec6_1.head())
print(len(df_rec6_1))

df_rec6_1


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率,所在地_都道府県_補完
240,A04122803,13_東京都,53,2018,289,0.0050,13_東京都
241,A04122803,NaN,53,2019,308,0.0025,13_東京都
244,A09104114,NaN,64,2018,289,0.0049,13_東京都
245,A09104114,NaN,64,2019,323,0.0009,13_東京都
176,A11123355,11_埼玉県,254,2018,349,0.0032,11_埼玉県


250


In [8]:
# {コード3.23 2019 年かつ売上高利益率0.5 ％以上の企業の抽出}
df_rec6_2=df_rec6_1[(df_rec6_1['売上高利益率'] >= 0.005) & (df_rec6_1['決算期年'] == 2019) ]

print('df_rec6_2'); display(df_rec6_2.head())
print(len(df_rec6_2))

df_rec6_2


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率,所在地_都道府県_補完
177,A11123355,NaN,254,2019,324,0.0063,11_埼玉県
265,A30933509,NaN,45,2019,298,0.0051,13_東京都
279,A32543083,NaN,153,2019,288,0.0077,13_東京都
283,A32654740,NaN,104,2019,256,0.0058,13_東京都
287,A32790644,NaN,161,2019,251,0.0075,13_東京都


24


In [9]:
# {コード3.24 都道府県所在地に川を含んだ企業の抽出}
df_rec7 = df_rec4[df_rec4['所在地_都道府県_補完'].str.contains('川')]

print('df_rec7'); display(df_rec7.head())
print(len(df_rec7))

df_rec7


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率,所在地_都道府県_補完
468,A12123384,17_石川県,173,2018,330,0.0037,17_石川県
469,A12123384,NaN,173,2019,344,0.0058,17_石川県
910,A22103080,37_香川県,0,2018,339,0.0011,37_香川県
911,A22103080,NaN,0,2019,289,0.0012,37_香川県
388,A25109297,14_神奈川県,175,2018,284,0.0036,14_神奈川県


66


In [10]:
# {コード3.25 レコードの累積処理}
df_rec8_1=df_rec4[df_rec4['決算期年'] == 2019]
    
df_rec8_2=df_rec8_1.assign(累計売上高=df_rec8_1['売上高_百万円'].cumsum())

print('df_rec8_2'); display(df_rec8_2.head())
print(len(df_rec8_2))

df_rec8_2


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率,所在地_都道府県_補完,累計売上高
803,A00115478,NaN,102,2019,258,0.0045,29_奈良県,258
921,A00116790,NaN,91,2019,315,0.0052,38_愛媛県,573
937,A03112202,NaN,88,2019,292,0.0039,40_福岡県,865
693,A04122570,NaN,103,2019,397,0.0033,27_大阪府,1262
241,A04122803,NaN,53,2019,308,0.0025,13_東京都,1570


517


In [11]:
# {コード3.26 従業員数の区分作成}
df_rec9=df_rec8_2.copy()
df_rec9['従業員区分'] = np.where((df_rec9['従業員数'] == 0), '1_0名or未詳',
                             np.where((df_rec9['従業員数'] <= 50), '2_1名～50名',
                                 np.where((df_rec9['従業員数'] <= 100), '3_51名～100名',
                                     np.where((df_rec9['従業員数'] <= 300), '4_101名～300名',
                                    '5_300名超'))))


print('df_rec9'); display(df_rec9.head())
print(len(df_rec9))

df_rec9


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率,所在地_都道府県_補完,累計売上高,従業員区分
803,A00115478,NaN,102,2019,258,0.0045,29_奈良県,258,4_101名～300名
921,A00116790,NaN,91,2019,315,0.0052,38_愛媛県,573,3_51名～100名
937,A03112202,NaN,88,2019,292,0.0039,40_福岡県,865,3_51名～100名
693,A04122570,NaN,103,2019,397,0.0033,27_大阪府,1262,4_101名～300名
241,A04122803,NaN,53,2019,308,0.0025,13_東京都,1570,3_51名～100名


517


In [12]:
# {コード3.27 区分別ランキングの付与}
df_rec9= df_rec9.sort_values(['決算期年','所在地_都道府県_補完','従業員区分','売上高利益率'],ascending=[True,True,True,False])

df_rec10=df_rec9.assign(都道府県従業員別売上高利益率ランキング=df_rec9.groupby(['所在地_都道府県_補完','従業員区分'])['売上高利益率'].rank(method='min',ascending=False))


print('df_rec10'); display(df_rec10.head())
print(len(df_rec10))

df_rec10


,企業管理コード,所在地_都道府県,従業員数,決算期年,売上高_百万円,売上高利益率,所在地_都道府県_補完,累計売上高,従業員区分,都道府県従業員別売上高利益率ランキング
33,A48982614,NaN,0,2019,253,0.0032,01_北海道,77948,1_0名or未詳,1.0
3,A08507655,NaN,0,2019,278,0.0026,01_北海道,6808,1_0名or未詳,2.0
15,A28144276,NaN,0,2019,311,-0.0009,01_北海道,31208,1_0名or未詳,3.0
35,A56231550,NaN,45,2019,287,0.0050,01_北海道,83445,2_1名～50名,1.0
27,A42128109,NaN,31,2019,332,0.0034,01_北海道,67462,2_1名～50名,2.0


517
